In [1]:
import Pkg; Pkg.add("DynamicPolynomials"); Pkg.add("OrderedCollections");
import Base: show
using DynamicPolynomials
using OrderedCollections

abstract type GeometricEntity end

struct Segment <: GeometricEntity
    l::Symbol
    r::Symbol
end
show(io::IO, AB::Segment) = print(io, AB.l, AB.r)
const segment = Segment

struct Circle <: GeometricEntity
    O::Symbol
    A::Symbol
end
show(io::IO, γ::Circle) = print(io, "circle center ", γ.O, " radius ", γ.O, γ.A)
const circle = Circle

abstract type GeometricConstraint end

struct Parallelism <: GeometricConstraint
    l::Segment
    r::Segment
end
const ∥ = Parallelism

struct Perpendicularity <: GeometricConstraint
    l::Segment
    r::Segment
end
const ⟂ = Perpendicularity

struct Congruence{T<:GeometricEntity} <: GeometricConstraint
    l::T
    r::T
end
const ≅ = Congruence

struct Appartenence{T<:GeometricEntity} <: GeometricConstraint
    l::Symbol
    r::T
end
Base.:∈(P::Symbol, obj::GeometricEntity) = Appartenence(P, obj)

for (op, sym) in [(Parallelism, :∥), (Perpendicularity, :⟂), (Congruence, :≅), (Appartenence{Segment}, :∈)]
    @eval show(io::IO, rel::$op) = print(io, rel.l, " ", $(QuoteNode(sym)), " ", rel.r)
end

show(io::IO, rel::Appartenence{Circle}) = print(io, rel.r.O, rel.l, " ≅ ", rel.r.O, rel.r.A)

struct Intersection{T <: GeometricEntity, S <: GeometricEntity} <: GeometricConstraint
    P::Symbol
    l::T
    r::S
end
const ∩ = Intersection

show(io::IO, rel::Intersection) = print(io, rel.P, " = ", rel.l, " ∩ ", rel.r)

struct Midpoint <: GeometricConstraint
    M::Symbol
    seg::Segment
end
Midpoint(M, A, B) = Midpoint(M, Segment(A, B))
const midpoint = Midpoint

show(io::IO, rel::Midpoint) = print(io, rel.M, " ∈ ", rel.seg, " ∧ ", rel.seg.l, rel.M, " ≅ ", rel.M, rel.seg.r)

struct Parallelogram <: GeometricConstraint
    A::Symbol
    B::Symbol
    C::Symbol
    D::Symbol
end
const parallelogram = Parallelogram
show(io::IO, rel::Parallelogram) = print(io, rel.A, rel.B, rel.C, rel.D, " parallelogram")

struct Projection <: GeometricConstraint
    H::Symbol
    P::Symbol
    seg::Segment
end
const ↓ = Projection
show(io::IO, rel::Projection) = print(io, rel.H, " ∈ ", rel.seg, " ∧ ", rel.P, rel.H, " ⟂ ", rel.seg)

struct CircleThreePoints <: GeometricConstraint
    O::Symbol
    A::Symbol
    B::Symbol
    C::Symbol
end
show(io::IO, rel::CircleThreePoints) = print(io, rel.O, rel.A, " ≅ ", rel.O, rel.B, " ≅ ", rel.O, rel.C)
Circle(O, A, B, C) = CircleThreePoints(O, A, B, C)

abstract type PointStatus end

struct Free <: PointStatus end
show(io::IO, ::Free) = print(io, "free")

struct SemiFree <: PointStatus
    eq1::Int
end
show(io::IO, sf::SemiFree) = print(io, "semifree by (", sf.eq1, ")")

struct Dependent <: PointStatus
    eq1::Int
    eq2::Int
end
Dependent(a) = Dependent(a, 0)
function show(io::IO, d::Dependent)
    print(io, "dependent by ($(d.eq1)$(iszero(d.eq2) ? "" : ", $(d.eq2)"))")
end

_update_status(::Free, ::Free) = Free()
_update_status(::Free, s::PointStatus) = s
_update_status(s::PointStatus, ::Free) = s
_update_status(s1::SemiFree, s2::SemiFree) = Dependent(s1.eq1, s2.eq1)

function _update_status(s1::PointStatus, s2::PointStatus)
    throw(ArgumentError("Cannot add statement ($(s2.eq1)) to point already defined as $s1"))
end

abstract type GeometricStatement end

struct Hypothesis <: GeometricStatement
    points::OrderedDict{Symbol, PointStatus}
    constraints::Vector{GeometricConstraint}
end

function show(io::IO, given::Hypothesis)
    println(io, "POINTS:")
    println(io, "------------")
    for (p, s) in given.points
        println(io, p, " : ", s)
    end
    println(io, "")
    println(io, "HYPOTHESIS:")
    println(io, "------------")
    for (i, c) in enumerate(given.constraints)
        println(io, "(", i, ") ", c)
    end
end


struct Thesis <: GeometricStatement
    constraints::Vector{GeometricConstraint}
end

function show(io::IO, claim::Thesis)
    println(io, "THESIS:")
    println(io, "------------")
    for c in claim.constraints
        println(io, c)
    end
end

macro given(block)
    _parse_given(block)
end

macro claim(block)
    _parse_th(block)
end

#####################
## Prover interface #
#####################

abstract type AbstractGeometricProver end
abstract type AbstractGeometricProof end

struct NoProof <: AbstractGeometricProof end
show(io::IO, ::NoProof) = println(io, "PROOF:\n------\nProof missing")
isproved(::NoProof) = false

prove(given::Hypothesis, claim::Thesis) = prove(given, claim, RittWuMethod)

#####################
# Theorem interface #
#####################
struct Theorem{T<:AbstractGeometricProof}
    given::Hypothesis
    claim::Thesis
    p::T
end

function show(io::IO, thm::Theorem)
    show(io, thm.given)
    println(io)
    show(io, thm.claim)
    println(io)
    show(io, thm.p)
end

Theorem(given::Hypothesis, claim::Thesis) = Theorem(given, claim, NoProof())

prove(t::Theorem, prover=RittWuMethod) = Theorem(t.given, t.claim, prove(t.given, t.claim, prover))

isproved(t::Theorem) = isproved(t.p)


sym2status(sym, i) = sym in (:∈, :⟂, :∥, :≅) ? SemiFree(i) : Dependent(i)

function _add_point!(dofs, P, s)
    if haskey(dofs, P)
        dofs[P] = _update_status(dofs[P], s)
    else
        dofs[P] = s
    end
end

function _parse_statement!(dofs, intros, ex)
    if ex.head == :(:=)
        newpoint = ex.args[1]
        constr = ex.args[2]
        newstatus = sym2status(constr.args[1], length(intros.args)+1)
        _add_free_points!(dofs, constr)
        _add_point!(dofs, newpoint, newstatus)
        _parse_constraint!(intros, constr, newpoint)
    elseif ex.head == :call && (ex.args[1] in (:Points, :points))
        for P in ex.args[2:end]
            _add_point!(dofs, P, Free())
        end
    end
end

function _parse_constraint!(intros, constr, newpoint=nothing)
    constr = _quotenodify(constr)
    if !isnothing(newpoint) && constr.args[1] in (:↓, :Midpoint, :midpoint, :∩, :Circle, :circle)
         insert!(constr.args, 2, QuoteNode(newpoint))
    end
    push!(intros.args, constr)
end

function _add_free_points!(dofs, ex::Expr)
    for s in ex.args[2:end]
        _add_free_points!(dofs, s)
    end
end

_add_free_points!(dofs, s::Symbol) = _add_point!(dofs, s, Free())

function _quotenodify(ex::Expr)
    for i in 2:length(ex.args)
        ex.args[i] = _quotenodify(ex.args[i])
    end
    return ex
end

_quotenodify(s::Symbol) = QuoteNode(s)

function _parse_given(block)
    dofs = OrderedDict{Symbol, PointStatus}()
    intros = :([])

    if block.head == :block
        for ex in block.args
            if ex isa Expr
                _parse_statement!(dofs, intros, ex)
            end
        end
    else
        _parse_statement!(dofs, intros, block)
    end
    return :(Hypothesis($dofs, $intros))
end

function _parse_th(block)
    claim = :([])

    if block.head == :block
        for ex in block.args
            if ex isa Expr
                _parse_constraint!(claim, ex)
            end
        end
    else
        if block isa Expr
            _parse_constraint!(claim, block)
        end
    end

    return :(Thesis($claim))
end


struct Coordinate{T, S}
    x::T
    y::S
end
function show(io::IO, c::Coordinate)
    print(io, "(")
    show(io, c.x)
    print(io, ", ")
    show(io, c.y)
    print(io, ")")
end

function compute_num_variables(given::Hypothesis)
    total_free = 0
    total_dep = 0
    for (P, s) in given.points
        total_free += numfree(s)
        total_dep += numdep(s)
    end
    return total_free, total_dep
end

numfree(::Free) = 2
numfree(::SemiFree) = 1
numfree(::Dependent) = 0
numdep(::Free) = 0
numdep(::SemiFree) = 1
numdep(::Dependent) = 2

function get_variables(given::Hypothesis)
    nu, nx = compute_num_variables(given)
    u, x = @polyvar u[1:nu-3] x[1:nx]
    return u, x
end

function assign_variables(given::Hypothesis)
    u, x = get_variables(given)
    point2var = OrderedDict{Symbol, Coordinate}()

    uidx = 1
    xidx = 1
    origin = true
    xaxis = true
    for (P, s) in given.points
        if s isa Free
            if origin
                point2var[P] = Coordinate(0, 0)
                origin = false
            elseif xaxis
                point2var[P] = Coordinate(u[uidx], 0)
                uidx += 1
                xaxis = false
            else
                point2var[P] = Coordinate(u[uidx], u[uidx+1])
                uidx += 2
            end
        elseif s isa SemiFree
            point2var[P] = Coordinate(u[uidx], x[xidx])
            uidx += 1
            xidx += 1
        elseif s isa Dependent
            point2var[P] = Coordinate(x[xidx], x[xidx+1])
            xidx += 2
        end
    end
    for (P, s) in given.points
        if s isa SemiFree
            eq = given.constraints[s.eq1]
            pol = algebry(eq, point2var)[1]
            if maxdegree(pol, point2var[P].y) == 0
                point2var[P] = Coordinate(point2var[P].y, point2var[P].x)
            end
        end
    end
    return point2var, x, u
end


function algebry(rel::Parallelism, coords)
    A = coords[rel.l.l]
    B = coords[rel.l.r]
    C = coords[rel.r.l]
    D = coords[rel.r.r]
    return ((B.x - A.x)*(D.y - C.y) - (B.y - A.y)*(D.x - C.x),)
end

function algebry(rel::Perpendicularity, coords)
    A = coords[rel.l.l]
    B = coords[rel.l.r]
    C = coords[rel.r.l]
    D = coords[rel.r.r]
    return ((B.x - A.x)*(D.x - C.x) + (B.y - A.y)*(D.y - C.y),)
end

function algebry(rel::Congruence{Segment}, coords)
    A = coords[rel.l.l]
    B = coords[rel.l.r]
    C = coords[rel.r.l]
    D = coords[rel.r.r]
    return ((B.x-A.x)^2 + (B.y-A.y)^2 - (D.x-C.x)^2 - (D.y-C.y)^2,)
end

function algebry(rel::Parallelogram, coords)
    A = coords[rel.A]
    B = coords[rel.B]
    C = coords[rel.C]
    D = coords[rel.D]
    return (B.x + D.x - A.x - C.x, B.y + D.y - A.y - C.y)
end

function algebry(rel::Midpoint, coords)
    M = coords[rel.M]
    A = coords[rel.seg.l]
    B = coords[rel.seg.r]
    return (2*M.x-A.x-B.x, 2*M.y-A.y-B.y)
end

function algebry(rel::Appartenence{Segment}, coords)
    P = coords[rel.l]
    A = coords[rel.r.l]
    B = coords[rel.r.r]
    return ((B.x - A.x)*(P.y - A.y) - (B.y - A.y)*(P.x - A.x),)
end

function algebry(rel::Intersection{Segment, Segment}, coords)
    return (algebry(Appartenence(rel.P, rel.l), coords)[1], algebry(Appartenence(rel.P, rel.r), coords)[1])
end

function algebry(rel::Projection, coords)
    return (algebry(Appartenence(rel.H, rel.seg), coords)[1], algebry(Perpendicularity(Segment(rel.P, rel.H), rel.seg), coords)[1])
end

function algebry(rel::Appartenence{Circle}, coords)
    OP = Segment(rel.r.O, rel.l)
    OA = Segment(rel.r.O, rel.r.A)
    return algebry(Congruence(OA, OP), coords)
end

function algebry(rel::CircleThreePoints, coords)
    OA = Segment(rel.O, rel.A)
    OB = Segment(rel.O, rel.B)
    OC = Segment(rel.O, rel.C)
    return (algebry(Congruence(OA, OB), coords)[1], algebry(Congruence(OA, OC), coords)[1])
end

function LC(p, var)
    d = maxdegree(p, var)
    d == 0 && return zero(p)
    return polynomial([subs(t, var => 1) for t in terms(p) if degree(t, var) == d])
end

function pseudorem(f, g, y)
    r = f
    lc = LC(g, y)
    m = maxdegree(g, y)
    d = maxdegree(r, y)

    while d ≥ m && !iszero(r)
        r = lc * r - LC(r, y) * g * y ^ (d - m)
        d = maxdegree(r, y)
    end

    return r
end

function triangulize(H, vars)
    n = length(H)

    T = similar(H)
    @inbounds for i = n:-1:1
        degrees = Int[] # degrees of polynomials in C
        indeces = Int[] # indeces in H of polynomials in C

        # initialize
        @inbounds for (idx, p) in enumerate(H)
            deg = maxdegree(p, vars[i])
            if deg > 0
                push!(degrees, deg)
                push!(indeces, idx)
            end
        end

        # If there are several polynomials in C but none has degree 1 in x, transform the
        # set until there is a polynomial of degree 1 or only one polynomial is left
        C = H[indeces]
        while minimum(degrees) > 1 && length(C) > 1
            _, idx_max = findmax(degrees)
            _, idx_min = findmin(degrees)

            # special case when all  have same degree
            if idx_max == idx_min
                idx_min = 1
                idx_max = 2
            end

            r = pseudorem(C[idx_max], C[idx_min], vars[i])
            d = maxdegree(r, vars[i])
            if iszero(d)
                H[indeces[idx_max]] = r
                deleteat!(C, idx_max)
                deleteat!(degrees, idx_max)
                deleteat!(indeces, idx_max)
            else
                C[idx_max] = r
                degrees[idx_max] = d
            end
        end

        # if only one polynomial in C, add to triangular set and go to the next variable
        if length(indeces) == 1
            T[i] = H[indeces[1]]
            deleteat!(H, indeces[1])
            continue
        end

        # if multiple polynomials but at least one has degree 1, add this to triangular set
        # and replace the others in H with the pseudoremainder
        idx = findlast(==(1), degrees)
        if !isnothing(idx)
            pivot = H[indeces[idx]]
            T[i] = pivot
            #pushfirst!(T, pivot)

            @inbounds for j in indeces
                if j != indeces[idx]
                    H[j] = pseudorem(H[j], pivot, vars[i])
                end
            end
            deleteat!(H, indeces[idx])
            continue
        end

    end
    return T
end

struct RittWuMethod <: AbstractGeometricProver end

mutable struct RittWuProof{S} <: AbstractGeometricProof
    coords::OrderedDict{Symbol, Coordinate}
    H::Vector{S}
    T::Vector{S}
    R::Matrix{S}
    u::Vector{PolyVar{true}}
    x::Vector{PolyVar{true}}
end

function show(io::IO, rwp::RittWuProof)
    println(io, "Assigned coordinates:")
    println(io, "---------------------")
    for (P, c) in rwp.coords
        println(io, P, " = ", c)
    end
    println(io)
    for (idx, p) in enumerate(eachcol(rwp.R))
        status = iszero(first(p)) ? "success" : "fail"
        println(io, "Goal $idx: $status")
    end
    println(io, "\nNondegeneracy conditions:")
    println(io, "-------------------------")
    for c in ndg(rwp)
        show(io, c)
        println(io, " ≠ 0")
    end
end
print("Using Package Bhumiti")
function prove(given::Hypothesis, claim::Thesis, ::Type{RittWuMethod}, coords, u, x)
    H = [cc for c in given.constraints for cc in algebry(c, coords) if !iszero(cc)]
    G = [cc for c in claim.constraints for cc in algebry(c, coords)]
    T = triangulize(copy(H), x)

    R = Matrix{eltype(T)}(undef, length(x) + 1, length(G))
    @inbounds for (idx, g) in enumerate(G)
        R[end, idx] = g
        @inbounds for i in length(x):-1:1
            R[i, idx] = pseudorem(R[i+1, idx], T[i], x[i])
        end
    end

    return RittWuProof(coords, H, T, R, u, x)
end

function prove(given::Hypothesis, claim::Thesis, ::Type{RittWuMethod})
    coords, x, u = assign_variables(given)
    return prove(given, claim, RittWuMethod, coords, u, x)
end

isproved(rwp::RittWuProof) = [iszero(first(p)) for p in eachcol(rwp.R)]
ndg(p::RittWuProof) = [LC(t, xi) for (t, xi) in zip(p.T, p.x) if maxdegree(LC(t, xi)) != 0]

    Updating registry at `/srv/julia/pkg/registries/General.toml`
   Resolving package versions...
   Installed DynamicPolynomials ────── v0.4.5
   Installed MultivariatePolynomials ─ v0.4.6
   Installed MutableArithmetics ────── v1.1.0
┌ Warning: The active manifest file at `/home/jovyan/Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.7/Pkg/src/manifest.jl:287
    Updating `~/Project.toml`
  [7c1d4256] + DynamicPolynomials v0.4.5
    Updating `~/Manifest.toml`
  [7c1d4256] + DynamicPolynomials v0.4.5
  [102ac46a] + MultivariatePolynomials v0.4.6
  [d8a4904e] + MutableArithmetics v1.1.0
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /home/jovyan/Manifest.toml:0
Precompil

Using Package Bhumiti

ndg (generic function with 1 method)

# Exaple 1
[![Screenshot-20221221-125914.png](https://i.postimg.cc/NF32CGXh/Screenshot-20221221-125914.png)](https://postimg.cc/4H13mgww)

In [2]:
h = @given begin
    points(A, B, C)
    C := Segment(A, B) ⟂ Segment(A, C)
    M₁ := Midpoint(A, B)
    M₂ := Midpoint(A, C)
    M₃ := Midpoint(B, C)
    H := A ↓ Segment(B, C)
    O := Circle(M₁, M₂, M₃)
end

POINTS:
------------
A : free
B : free
C : semifree by (1)
M₁ : dependent by (2)
M₂ : dependent by (3)
M₃ : dependent by (4)
H : dependent by (5)
O : dependent by (6)

HYPOTHESIS:
------------
(1) AB ⟂ AC
(2) M₁ ∈ AB ∧ AM₁ ≅ M₁B
(3) M₂ ∈ AC ∧ AM₂ ≅ M₂C
(4) M₃ ∈ BC ∧ BM₃ ≅ M₃C
(5) H ∈ BC ∧ AH ⟂ BC
(6) OM₁ ≅ OM₂ ≅ OM₃


In [3]:
g = @claim H ∈ Circle(O, M₁)

THESIS:
------------
OH ≅ OM₁


In [4]:
prove(h,g)

Assigned coordinates:
---------------------
A = (0, 0)
B = (u₁, 0)
C = (x₁, u₂)
M₁ = (x₂, x₃)
M₂ = (x₄, x₅)
M₃ = (x₆, x₇)
H = (x₈, x₉)
O = (x₁₀, x₁₁)

Goal 1: success

Nondegeneracy conditions:
-------------------------
u₁ ≠ 0
-u₁² + 2u₁x₁ - u₂² - x₁² ≠ 0
u₂ ≠ 0
4x₂x₅ - 4x₂x₇ - 4x₃x₄ + 4x₃x₆ + 4x₄x₇ - 4x₅x₆ ≠ 0
-2x₃ + 2x₇ ≠ 0


![](https://th.bing.com/th/id/R.e210f6f13bcd6ade8fc64b14f0fba4a6?rik=N9aslUuLcN8ksQ&riu=http%3a%2f%2fwww.maths.gla.ac.uk%2fwws%2fcabripages%2fgraphics%2fpappus.gif&ehk=utzmcvL9ScJfKt1lSjEjZ%2bdg0%2b2jZGZorF5imP5cMCQ%3d&risl=&pid=ImgRaw&r=0)

In [6]:
h = @given begin
    points(A, B, C, D, E, F, P, Q, R)
    B := B ∈ Segment(A, C)
    E := E ∈ Segment(D, F)
    P := P ∈ Segment(A, E)
    P := P ∈ Segment(B, D)
    Q := Q ∈ Segment(A, F)
    Q := Q ∈ Segment(C, D)
    R := R ∈ Segment(B, F)
    R := R ∈ Segment(C, E)
end

POINTS:
------------
A : free
B : semifree by (1)
C : free
D : free
E : semifree by (2)
F : free
P : dependent by (3, 4)
Q : dependent by (5, 6)
R : dependent by (7, 8)

HYPOTHESIS:
------------
(1) B ∈ AC
(2) E ∈ DF
(3) P ∈ AE
(4) P ∈ BD
(5) Q ∈ AF
(6) Q ∈ CD
(7) R ∈ BF
(8) R ∈ CE


In [7]:
g = @claim Q ∈ Segment(P, R)

THESIS:
------------
Q ∈ PR


In [8]:
prove(h, g)

Assigned coordinates:
---------------------
A = (0, 0)
B = (u₁, x₁)
C = (u₂, 0)
D = (u₃, u₄)
E = (u₅, x₂)
F = (u₆, u₇)
P = (x₃, x₄)
Q = (x₅, x₆)
R = (x₇, x₈)

Goal 1: success

Nondegeneracy conditions:
-------------------------
u₂ ≠ 0
-u₃ + u₆ ≠ 0
u₁x₂ - u₃x₂ + u₄u₅ - u₅x₁ ≠ 0
-u₁ + u₃ ≠ 0
u₂u₇ - u₃u₇ + u₄u₆ ≠ 0
-u₂ + u₃ ≠ 0
-u₁x₂ + u₂u₇ - u₂x₁ - u₅u₇ + u₅x₁ + u₆x₂ ≠ 0
-u₂ + u₅ ≠ 0


In [9]:
h = @given begin
    C := Segment(A, B) ∥ Segment(C, D)
    M := M ∈ Segment(A, B)
    N := N ∈ Segment(A, B)
    O := M ↓ Segment(C, D)
    P := N ↓ Segment(C, D) 
end

POINTS:
------------
A : free
B : free
C : semifree by (1)
D : free
M : semifree by (2)
N : semifree by (3)
O : dependent by (4)
P : dependent by (5)

HYPOTHESIS:
------------
(1) AB ∥ CD
(2) M ∈ AB
(3) N ∈ AB
(4) O ∈ CD ∧ MO ⟂ CD
(5) P ∈ CD ∧ NP ⟂ CD


In [10]:
g = @claim Segment(M, O) ≅ Segment(N, P)

THESIS:
------------
MO ≅ NP


In [11]:
prove(h, g)

Assigned coordinates:
---------------------
A = (0, 0)
B = (u₁, 0)
C = (u₂, x₁)
D = (u₃, u₄)
M = (u₅, x₂)
N = (u₆, x₃)
O = (x₄, x₅)
P = (x₆, x₇)

Goal 1: success

Nondegeneracy conditions:
-------------------------
-u₁ ≠ 0
u₁ ≠ 0
u₁ ≠ 0
-u₂² + 2u₂u₃ - u₃² - u₄² + 2u₄x₁ - x₁² ≠ 0
u₄ - x₁ ≠ 0
-u₂² + 2u₂u₃ - u₃² - u₄² + 2u₄x₁ - x₁² ≠ 0
u₄ - x₁ ≠ 0


In [13]:
h = @given begin
    D := Segment(A, B) ≅ Segment(C, D)
    D := Segment(A, C) ≅ Segment(B, D)
    C := Segment(A, B) ≅ Segment(A, C)
    E :=  Segment(A,D) ∩ Segment(B, C)
end

POINTS:
------------
A : free
B : free
C : semifree by (3)
D : dependent by (1, 2)
E : dependent by (4)

HYPOTHESIS:
------------
(1) AB ≅ CD
(2) AC ≅ BD
(3) AB ≅ AC
(4) E = AD ∩ BC


In [14]:
g = @claim Segment(A, D) ⟂ Segment(C, B)

THESIS:
------------
AD ⟂ CB


In [15]:
prove(h,g)

Assigned coordinates:
---------------------
A = (0, 0)
B = (u₁, 0)
C = (u₂, x₁)
D = (x₂, x₃)
E = (x₄, x₅)

Goal 1: success

Nondegeneracy conditions:
-------------------------
2u₁ - 2u₂ ≠ 0
u₁x₃ - u₂x₃ + x₁x₂ ≠ 0
-u₁ + u₂ ≠ 0
